# Load Video from URL

Import video files from URLs directly into Pixeltable tables.

**What's in this recipe:**
- Load video files from URLs into Pixeltable
- Extract video metadata and properties
- Work with video files in Pixeltable


## Problem

You have video files hosted at URLs and need to import them into Pixeltable to apply AI models, extract frames, or run video analysis.


## Solution

**What's in this recipe:**
- Load video files from URLs directly into Pixeltable tables
- Automatic schema handling for video data types
- Extract metadata like duration, frame rate, and resolution

You can load video files from URLs directly into Pixeltable tables. This allows you to immediately apply AI models and query results.

You can iterate on transformations before adding them to your table. Use `.select()` with `.collect()` to preview results on sample data—nothing is stored in your table. If you want to collect only the first few rows, use `.head(n)` instead of `.collect()`. Once you're satisfied, use `.add_computed_column()` to apply transformations to all rows in your table.


### Setup


In [ ]:
!uv add pixeltable

In [ ]:
import pixeltable as pxt

### Load Video from URL

We'll load the Jane Goodall video from Google Cloud Storage.


In [ ]:
# Create directory for video data
pxt.drop_dir('jane_goodall_video', force=True)
pxt.create_dir('jane_goodall_video')

### Create Pixeltable Table

Create a table with a video column and insert the video URL.


In [ ]:
# Create table with video column
t = pxt.create_table(
    'jane_goodall_video.clips',
    schema={
        'video': pxt.Video
    },
    comment='Jane Goodall video from Google Cloud Storage for video processing demonstrations'
)

In [ ]:
# Insert the video URL
t.insert([
    {
        'video': 'https://storage.googleapis.com/cloud-samples-data/video/JaneGoodall.mp4'
    }
])

In [ ]:
# View the table
t.select(t.video).head()


### Extract Video Frames

Use Pixeltable's built-in frame iterator to extract frames from the video.


In [ ]:
# Create a view with frame-level data using Pixeltable's built-in FrameIterator
# Extract frames at 1 fps (1 frame per second)
from pixeltable.iterators import FrameIterator

frames = pxt.create_view(
    'jane_goodall_video.frames',
    t,
    iterator=FrameIterator.create(video=t.video, fps=1)
)

In [ ]:
# View extracted frames
frames.select(frames.pos, frames.frame).limit(10).collect()

### Transcribe Audio with OpenAI

Extract and transcribe the audio from the video using OpenAI's Whisper model.


In [ ]:
# Extract audio and transcribe with OpenAI Whisper
from pixeltable.functions import openai

t.add_computed_column(audio=t.video.extract_audio())
t.add_computed_column(transcription=openai.transcriptions(audio=t.audio, model='whisper-1'))

In [ ]:
# View the transcription
t.select(t.transcription).head()

### Transcribe Audio with OpenAI

Extract and transcribe the audio from the video using OpenAI's Whisper model.


In [ ]:
# Extract audio and transcribe with OpenAI Whisper
from pixeltable.functions import openai

t.add_computed_column(audio=t.video.extract_audio())
t.add_computed_column(transcription=openai.transcriptions(audio=t.audio, model='whisper-1'))

In [ ]:
# View the transcription
t.select(t.transcription).head()

### Create Sentence-Level View

Split the transcription into sentences with timestamps.


In [ ]:
# Create a view with sentence-level data using Pixeltable's built-in sentence splitter
sentences = pxt.create_view(
    'jane_goodall_video.sentences',
    t,
    iterator=t.transcription.sentences()
)

In [ ]:
# View sentence-level transcriptions
sentences.select(sentences.text, sentences.start, sentences.end).head(10)

### Create Frame-Level View

Extract individual frames from the video at regular intervals.


In [ ]:
# Create a view with frame-level data using Pixeltable's built-in frame iterator
# Extract frames at 1 fps (1 frame per second)
frames = pxt.create_view(
    'jane_goodall_video.frames',
    t,
    iterator=t.video.frame_iterator(fps=1)
)

In [ ]:
# View extracted frames
frames.select(frames.pos, frames.frame).head(10)

## See also

- [Working with Video in Pixeltable](https://docs.pixeltable.com/docs/working-with-video)
- [Scene Detection](https://docs.pixeltable.com/api/functions/video#scene_detect)
- [OpenAI Transcriptions](https://docs.pixeltable.com/api/functions/openai#transcriptions)
- [Custom UDFs and Iterators](https://docs.pixeltable.com/docs/user-defined-functions)
